# RAM

In [13]:
%%bash

cat <<EOF > ram/ram.vhdl
LIBRARY ieee;
USE ieee.std_logic_1164.all;
USE ieee.numeric_std.all;

entity ram IS
	PORT (
		clk : IN STD_LOGIC;
		a : IN STD_LOGIC_VECTOR(4 DOWNTO 0);
		di : IN STD_LOGIC_VECTOR(3 DOWNTO 0);
		we : IN STD_LOGIC;
		do : OUT STD_LOGIC_VECTOR(3 DOWNTO 0)
	);
END ram;

ARCHITECTURE Behavioral OF ram IS
	TYPE ram_type IS ARRAY (31 DOWNTO 0)
	OF STD_LOGIC_VECTOR (3 DOWNTO 0);
	SIGNAL mem : ram_type := (OTHERS => (OTHERS => '0'));
	SIGNAL read_a : STD_LOGIC_VECTOR(4 DOWNTO 0);
BEGIN
	PROCESS (clk)
	BEGIN
		IF rising_edge(Clk) THEN
			IF (we = '1') THEN
				mem(to_integer(unsigned(a))) <= di;
			END IF;
			read_a <= a;
		END IF;
	END PROCESS;
	do <= mem(to_integer(unsigned(read_a)));
END Behavioral;
EOF

In [ ]:
%%bash
cat <<EOF > ram/ram_tb.vhdl
LIBRARY IEEE;
USE IEEE.STD_LOGIC_1164.ALL;

ENTITY ram_tb IS
END ram_tb;

ARCHITECTURE behaviour OF ram_tb IS
  --  Declaration of the component that will be instantiated.
  COMPONENT ram
    PORT ( clk : IN STD_LOGIC;
          a : IN STD_LOGIC_VECTOR(4 DOWNTO 0);
          di : IN STD_LOGIC_VECTOR(3 DOWNTO 0);
          we : IN STD_LOGIC;
          do : OUT STD_LOGIC_VECTOR(3 DOWNTO 0));
  END COMPONENT;

  COMPONENT heartbeat
    PORT (clk : OUT STD_LOGIC);
  END COMPONENT;

  --  Specifies which entity is bound with the component.
  FOR ram_0 : ram USE ENTITY work.ram;
  FOR heartbeat_0 : heartbeat USE ENTITY work.heartbeat;

  SIGNAL clk : STD_LOGIC;
  SIGNAL a : STD_LOGIC_VECTOR(4 DOWNTO 0) := "00000";
  SIGNAL di : STD_LOGIC_VECTOR(3 DOWNTO 0) := "0000";
  SIGNAL we : STD_LOGIC;
  SIGNAL do : STD_LOGIC_VECTOR(3 DOWNTO 0);

BEGIN
  --  Component instantiation.
  ram_0 : ram PORT MAP(clk => clk, a => a, di => di, we => we, do => do);
  heartbeat_0 : heartbeat PORT MAP(clk => clk);

  --  This process does the real job.
  PROCESS
    BEGIN
    wait for 100 ns;
    a <= "00000";  -- Address 0
    di <= "0001";  -- Data to write
    we <= '1';     -- Write enable
    WAIT for 100 ns;
    a <= "00001";  -- Address 0
    we <= '0';     -- Disable write
    WAIT for 100 ns;
    a <= "00000";  -- Address 0
  Wait;
  END PROCESS;

END ARCHITECTURE behaviour;
EOF

In [15]:
%%bash
cd ram
rm -f work* *.ghw
ghdl -a heartbeat.vhdl
ghdl -a ram.vhdl
ghdl -a ram_tb.vhdl
ghdl -r ram_tb --stop-time=5000ns --wave=sm1_tb.ghw

../../src/ieee/v93/numeric_std-body.vhdl:2098:7:@0ms:(assertion warning): NUMERIC_STD.TO_INTEGER: metavalue detected, returning 0
ghdl:info: simulation stopped by --stop-time @5us


Clean up cell

In [16]:
%%bash
cd ram
rm -f work* *.ghw